# 0. Marking.

***IMPORTANT***: Save a copy of this notebook into your Drive before you start.
- Please attempt all the questions marked for your group (Part II ✅ | Part III/MPhil ✅).
- Continue to part 2 after you are done with this one.

Please submit a zip file, containing both parts, consiting of of:
1. A text file with a publicly visible link to your notebooks in Google Colab or GitHub.
2. A downloaded copy (ipynb) of your notebooks or your zipped cloned GitHub repo. You may treat these as a report: we will not be re-executing the code you used to produce the answers unless required.

If you find yourself enjoying the material, feel free to attempt more! Provide your answers in a new cell below the question cell.

### 1. Introduction.


Welcome to the first lab in our Federated Learning (FL) course. This series of labs intends to introduce you to the practice of constructing Federated Learning systems. The most common form of FL, cross-device FL, is concerned with collaboratively training models on edge devices or “clients'' encapsulating their private data. As such, FL may:
- Reduce communication costs by only sharing model parameters.
- Distribute computation across a pool of previously unused devices.
- Partially preserve privacy by default.

A typical server-client FL system maintains a centralised server that synchronises training across iterations of rounds---the FL equivalent of epochs. A round generally proceeds as follows:
1. At the beginning of a round, the server sends a copy of the federated model to each client.
2. The clients then train their copy of the model on the private data they hold and send the post-training model updates to the server.
3. The server then aggregates all the client models to form a single federated model update which it applies before the next round. The most common means of aggregation is Federated Averaging which merges model parameters via simple weighted averaging.


![picture](https://drive.google.com/uc?id=1QhUB6Eb-P1hf0LCKWp5f6rEeGThLCmbE)


To a more significant extent than other ML fields, FL has historically been driven by practical considerations rather than theoretical results. Some main practical concerns in the field are related to the following:
- Communication costs involved in training models in a highly distributed fashion across edge devices.
- The underlying non-standardized hardware of clients within the federated network.
- Highly divergent data distributions across clients.

Such issues cannot be easily corrected, given FL's privacy and scale constraints, as persistent data on specific clients may not be trackable. In a worst-case setting, a single client may only participate in training once for cross-device FL with millions of potential devices.

In this lab, we will use a realistic FL dataset to exemplify how such problems arise when ML is taken from a centralized or standard distributed ML setting to a federated one. Specific questions we will look at include the following:
- How centralized ML can be modeled as FL with a single client and how to build a client abstraction capable of both.
- How the federated and local optimization objectives of FL interact between each other and how different settings on the server/client side may impact convergence.

To do so, we shall use the [Flower](https://flower.dev/) FL framework. The design of Flower is meant to allow for a fairly direct mapping between simulation and production FL scenarios.



![picture](https://drive.google.com/uc?id=19MZ2nHIgLTZEqgSgbP9z9BKVyiR8GT3X)


# 2. Building a client.



Rather than beginning with a complete standard centralised ML baseline, we will treat centralised ML as a particular case of FL--a single client holding the entire centralised dataset.

For the rest of our labs, a client shall represent an abstraction that:
- Encapsulates the data available on a particular device involved in FL.
- Trains a model it receives on that local (train) dataset.
- Can test models it receives on its local (test) dataset.

By the end of this section, you will have constructed such a client abstraction and trained a model on a full-scale dataset. Later work shall merely involve connecting multiple such clients in a federated network using Flower.

To begin, we will need to install some dependencies.

In [ ]:
# The simulation component of the flower framework uses RAY under the hood.
# `pip` could produce some errors. Do not worry about them.
# The execution has been verified; it's working anyway.
! pip install --quiet --upgrade "pip"
! pip install --quiet git+https://github.com/Iacob-Alexandru-Andrei/flower.git@teaching torch torchvision matplotlib gdown tqdm seaborn ray=="2.6.3"
# The following is just needed to show the folder tree
! apt-get install -qq tree

Our first client abstraction shall be as simple as possible and will require adjustment to match the structure that the framework expects. However, it shall be conceptually identical and require only light API changes.

In [ ]:
## Imports

import csv
import numbers
import os
import random
from collections import OrderedDict, defaultdict
from copy import deepcopy
from pathlib import Path
from typing import *

import flwr as fl
import gdown
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from flwr.common import Metrics
from flwr.common.parameter import ndarrays_to_parameters
from flwr.common.typing import NDArrays, Parameters, Scalar
from flwr.server import ServerConfig
from flwr.server.strategy import FedAvgM as FedAvg
from PIL import Image
from PIL.Image import Image as ImageType
from torch.nn import Module
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from tqdm import tqdm
from enum import IntEnum


# Add new seeds here for easy autocomplete
class Seeds(IntEnum):
    DEFAULT = 1337


np.random.seed(Seeds.DEFAULT)
random.seed(Seeds.DEFAULT)
torch.manual_seed(Seeds.DEFAULT)
torch.backends.cudnn.benchmark = False  # type: ignore
torch.backends.cudnn.deterministic = True  # type: ignore


def convert(o):
    if isinstance(o, np.int64) or isinstance(o, np.int32):
        return int(o)
    if isinstance(o, np.float32) or isinstance(o, np.float64):
        return float(o)
    raise TypeError


def fit_client_seeded(client, params, conf, seed=Seeds.DEFAULT, **kwargs):
    """Wrapper to always seed client training."""
    np.random.seed(seed)
    torch.manual_seed(seed)
    random.seed(seed)
    return client.fit(params, conf, **kwargs)


PathType = Optional[Union[Path, str]]


def get_device() -> str:
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda"
    elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
        device = "mps"
    return device

In [ ]:
class SimpleClient:
    def __init__(self, cid: int, partition_dir: Path) -> None:
        """Function to initialise the client with its unique id
        and the directory from which it can load its data.

        Args:
            cid (int): Unique client id for a client used to map it to its data partition
            partition_dir (Path): The directory containing data for each client/client id
        """
        self.cid: str = str(cid)
        self.partition_dir: Path = partition_dir
        self.device: str = get_device()

    def fit(self, net: Module, config: Dict[str, Any]) -> Tuple[Module, int, dict]:
        """Function which receives and trains a model on the local client data using parameters from the config dict

        Args:
            net (Module): Pytorch model
            config (Dict[str, Any]): Dictionary describing the training parameters

        Returns:
            Tuple[Module, int, dict]: Returns the updated model, the size of the local dataset and (optionally) other metrics
        """
        net = deepcopy(net)
        net.to(self.device)
        train_loader: DataLoader = self._create_data_loader(config, name="train")
        self._train(net, train_loader=train_loader, config=config)
        return net, len(train_loader), {}

    def evaluate(self, net: Module, config: Dict[str, Any]) -> Tuple[float, int, dict]:
        """Function which receives and tests a model on the local client data using parameters from the config dict

        Args:
            net (Module): Pytorch model
            config (Dict[str, Any]): Dictionary describing the test parameters

        Returns:
            Tuple[float, int, dict]: Returns the loss accumulate during test, the size of the local dataset and other metrics such as accuracy
        """
        net = deepcopy(net)
        net.to(self.device)
        test_loader: DataLoader = self._create_data_loader(config, name="test")
        loss, accuracy = self._test(net, test_loader=test_loader, config=config)
        return loss, len(test_loader), {"local_accuracy": accuracy}

    def _create_data_loader(self, config: Dict[str, Any], name: str) -> DataLoader:
        """Creates the data loader using the specified config parameters

        Args:
            config (Dict[str, any]): Dictionary containing dataloader and dataset parameters
            name (str): Load the training or testing set for the client

        Returns:
            DataLoader: A pytorch dataloader iterable for training/testing
        """
        batch_size: int = config["batch_size"]
        num_workers: int = config["num_workers"]
        dataset = self._load_dataset(name, config)
        return DataLoader(
            dataset=dataset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=num_workers,
            drop_last=False if name == "test" else True,
        )

    # All methods from here on are task-specific
    # And need to be implemented
    def _load_dataset(self, name: str, config: Dict[str, Any]) -> Dataset:  # type: ignore
        pass

    def _train(
        self, net: Module, train_loader: DataLoader, config: Dict[str, Any]
    ) -> float:  # type: ignore
        pass

    def _test(
        self, net, test_loader: DataLoader, config: Dict[str, Any]
    ) -> Tuple[float, float]:  # type: ignore
        pass

This client covers all the previously stated functionality; it can:
- Associate a given dataset/dataset partition to itself based on the `data_dir` and `cid` combination
- Train any model provided to its `fit` function on the local training set
- Evaluate any model on the local test set.

To get this client training, we need to fill in the necessary gaps: the dataset loading procedure, the model, the training procedure and the testing procedure.

**Question 1 (Part II ✅ | Part III/MPhil ✅):**

(These are meant to be conceptual questions. You should provide written answers for these. **No more than 3 sentences each**. **No code** is needed)

1. What is the time complexity of transmitting models to-and-from the client?
2. Give at least two examples of machine learning and data science techniques you cannot apply since data is kept private on single clients.

# 3. Loading the federated dataset.

We have chosen to use the FEMNIST dataset originally published in [LEAF](https://arxiv.org/abs/1812.01097) for all of our labs due to its naturally heterogeneous properties. Most FL examples rely on a centralised dataset such as CIFAR-10, which is artificially partitioned to fit an FL context. FEMNIST is an image dataset with 28×28 greyscale images split into 62 distinct classes. We shall initially treat it as a centralised dataset. However, its federated nature will be helpful later on.

It is, by default, partitioned based on the creator of a specific image, thus modelling a realistic data-generation scenario. A complete list of its statistics may be found on the [LEAF Website](https://leaf.cmu.edu/).

We will extensively discuss federated datasets during the next lab. In this session, we will skip such discussions to spend more time on “federating” a centralised model. The following few cells will provide all the relevant functions and objects to load the FEMNIST dataset. Also, some **optional** insights will be given.

In [ ]:
home_dir = content if (content := Path("/content")).exists() else Path.cwd()
dataset_dir: Path = home_dir / "femnist"
data_dir: Path = dataset_dir / "data"
centralized_partition: Path = dataset_dir / "client_data_mappings" / "centralized"
centralized_mapping: Path = dataset_dir / "client_data_mappings" / "centralized" / "0"
federated_partition: Path = dataset_dir / "client_data_mappings" / "fed_natural"

#  Download compressed dataset
if not (home_dir / "femnist.tar.gz").exists():
    id = "1-CI6-QoEmGiInV23-n_l6Yd8QGWerw8-"
    gdown.download(
        f"https://drive.google.com/uc?export=download&confirm=pbef&id={id}",
        str(home_dir / "femnist.tar.gz"),
    )

# Decompress dataset
if not dataset_dir.exists():
    !tar -xf {str(home_dir)}/femnist.tar.gz -C {str(home_dir)} 2> /dev/null
    print(f"Dataset extracted in {dataset_dir}")

You can **optionally** execute the following cell to look at the folder tree induced by unzipping the compressed dataset.

In [ ]:
# Showing resulting folder tree
! tree -dC -L 3 ./

For those of you gifted with eager curiosity, you can find the [Jupyter notebook](https://drive.google.com/file/d/1-I0uPPzm1ONlLD-u4XCFGqUgz6KPJzDe/view?usp=share_link) used to create this dataset from the version publicly available from TensorFlow Federated.

We have thus downloaded two different partitions of the FEMNIST dataset. We will first use the “centralised” partition, in which we have distributed all the samples to one client. The second is the “natural” partition, in which the images/samples have been divided by writers.

Samples have been collected once in the `data` folder, composed of three subfolders- `train`, `test` and `val`-, each containing one file for each sample in those sets. Then, it is the `client_data_mappings` folder that contains the relevant structure, composed of subfolders and `.csv` files, each describing the partitions. In fact, for each client, a `train.csv` and a `test.csv` are provided, containing a row for each sample in the client set that reports `client_id`, `sample_path`, `sample_id`, `sample_label`. It is worth mentioning that while train and test sets have been composed of the same clients, the validation set has been composed of different clients. That is why the only `val.csv` contained in this folder structure is under the centralised partition.

The number of clients composing the train and test sets is 3230, while the number of clients composing the `val` set is 170. The splitting has been chosen to make the validation step quick.

# 4. Centralized ML

We will begin with our centralised “partition”. We then use the entire `train.csv` and `test.csv` contained in `client_data_mappings/centralized/0`. For a large section of FL tasks, the centralised baseline represents the upper limit of performance.

In [ ]:
class FEMNIST(Dataset):
    def __init__(
        self,
        mapping: Path,
        data_dir: Path = data_dir,
        name: str = "train",
        transform: Optional[Callable[[ImageType], Any]] = None,
        target_transform: Optional[Callable[[int], Any]] = None,
    ):
        """Function to initialise the FEMNIST dataset.

        Args:
            mapping (Path): path to the mapping folder containing the .csv files.
            data_dir (Path): path to the dataset folder. Defaults to data_dir.
            name (str): name of the dataset to load, train or test.
            transform (Optional[Callable[[ImageType], Any]], optional): transform function to be applied to the ImageType object. Defaults to None.
            target_transform (Optional[Callable[[int], Any]], optional): transform function to be applied to the label. Defaults to None.
        """
        self.data_dir = data_dir
        self.mapping = mapping
        self.name = name

        self.data: Sequence[Tuple[str, int]] = self._load_dataset()
        self.transform: Optional[Callable[[ImageType], Any]] = transform
        self.target_transform: Optional[Callable[[int], Any]] = target_transform

    def __getitem__(self, index) -> Tuple[Any, Any]:
        """Function used by PyTorch to get a sample.

        Args:
            index (_type_): index of the sample.

        Returns:
            Tuple[Any, Any]: couple (sample, label).
        """
        sample_path, label = self.data[index]

        # Convert to the full path
        full_sample_path: Path = self.data_dir / self.name / sample_path

        img: ImageType = Image.open(full_sample_path).convert("L")

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            label = self.target_transform(label)

        return img, label

    def __len__(self) -> int:
        """Function used by PyTorch to get the length of the dataset as the number of samples.

        Returns:
            int: the length of the dataset.
        """
        return len(self.data)

    def _load_dataset(self) -> Sequence[Tuple[str, int]]:
        """Load the paths and labels of the partition
        Preprocess the dataset for faster future loading
        If opened for the first time.

        Raises:
            ValueError: raised if the mapping file does not exist.

        Returns:
            Sequence[Tuple[str, int]]: partition asked as a sequence of couples (path_to_file, label)
        """
        preprocessed_path: Path = (self.mapping / self.name).with_suffix(".pt")
        if preprocessed_path.exists():
            return torch.load(preprocessed_path)
        else:
            csv_path = (self.mapping / self.name).with_suffix(".csv")
            if not csv_path.exists():
                raise ValueError(f"Required files do not exist, path: {csv_path}")
            else:
                with open(csv_path, mode="r") as csv_file:
                    csv_reader = csv.reader(csv_file)
                    # Ignore header
                    next(csv_reader)

                    # Extract the samples and the labels
                    partition: Sequence[Tuple[str, int]] = [
                        (sample_path, int(label_id))
                        for _, sample_path, _, label_id in csv_reader
                    ]

                    # Save for future loading
                    torch.save(partition, preprocessed_path)
                    return partition

In [ ]:
dataset: FEMNIST = FEMNIST(mapping=centralized_mapping, data_dir=data_dir, name="train")

# Show random value
img, _ = dataset[4]
img

Given this fully “centralised” dataset, the client abstraction only requires functions implementing data loading, model training and testing.

In [ ]:
# Load with appropriate transforms
def to_tensor_transform(p: Any) -> torch.Tensor:
    """Transform the object given to a PyTorch Tensor.

    Args:
        p (Any): object to transform.

    Returns:
        torch.Tensor: resulting PyTorch Tensor
    """
    return torch.tensor(p)


def load_FEMNIST_dataset(mapping: Path, name: str) -> Dataset:
    """Function to load the FEMNIST dataset given the mapping .csv file.
    The relevant transforms are automatically applied.

    Args:
        mapping (Path): path to the mapping .csv file chosen.
        name (str): name of the dataset to load, train or test.

    Returns:
        Dataset: FEMNIST dataset object, ready-to-use.
    """
    transform = transforms.Compose(
        [
            # transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
        ]
    )

    return FEMNIST(
        mapping=mapping,
        name=name,
        data_dir=data_dir,
        transform=transform,
        target_transform=to_tensor_transform,
    )

Partitions in FL datasets may vary in data-per-client by orders of magnitude. While in the “centralised” case, one client holds $O(N)$ samples with $N \approx 80,5263$, a natural partitioning by image creator ID will result in each client holding $O(m)$ samples with $m \approx 226$.

This points to a more fundamental issue in FL, the unbalanced size of local datasets across clients. Several questions arise when considering the potential orders-of-magnitude gap between clients:
1. How do we set the amount of training each client does per round?
2. How should we balance the contribution of several clients, given differences in dataset size?

The original paper introducing FL, [Communication-Efficient Learning of Deep Networks from Decentralized Data](https://arxiv.org/abs/1602.05629), and the Federated Averaging algorithm we shall use today answers the questions in the following ways:
1. Set a number of local epochs homogenous across clients; each client iterates over their entire dataset for the provided number of epochs.
2. When combining the models of different clients, weigh them by the size of the local dataset of the clients.

For now, we shall choose to keep the local epoch design and place an absolute upper limit on the number of batches seen during training/testing due to the computational constraints of training the centralised client.

In [ ]:
max_train_batches_per_epoch: int = 100
max_test_batches_per_epoch: int = 100

**Question 2 (Part II ✅ | Part III/MPhil ✅):**

(This is meant to be a conceptual question. You should provide written answers for this. **No more than 3 sentences**. **No code** is needed)

1. Consider the three following scenarios for a population of five clients and identify how weighing client updates by the number of samples compares to unweighted averaging:
  -  Clients have the same number of samples
  -  One client has 50% of the samples, and they are of high quality (e.g., coherent English text)
  - One client has 50% of the samples, and they are of very low quality (e.g, incomprehensible mashup of letters)




**Question 3 (Part III/MPhil ✅):**

(This is meant to be a conceptual question. You should provide written answers for this. **No more than 3 sentences**. **No code** is needed)
1. In the previous three scenarios, what would happen if we set a homogenous number of training steps for all clients instead of epochs? I.e., clients loop over their dataset until the precise moment when they have processed N total samples.

While FL may use any training function that centralised ML can, there is an extra consideration regarding how much information a client may leak regarding their local dataset. For example, the model may contain enough data to recreate local samples perfectly under the right circumstances unless defensive measures are employed.

Later labs and lectures will explore several methods for mitigating such concerns. Until then, the three following pieces of information are currently considered acceptable from a privacy standpoint:
1. Model weights.
2. The size of the local dataset.
3. Cumulative loss values.

In [ ]:
def train_FEMNIST(
    net: Module,
    train_loader: DataLoader,
    epochs: int,
    device: str,
    optimizer: torch.optim.Optimizer,
    criterion: Module,
) -> float:
    """Trains the network on the training set.

    Args:
        net (Module): generic module object describing the network to train.
        train_loader (DataLoader): dataloader to iterate during the training.
        epochs (int): number of epochs of training.
        device (str): device name onto which perform the computation.
        optimizer (torch.optim.Optimizer): optimizer object.
        criterion (Module): generic module describing the loss function.

    Returns:
        float: the final epoch mean train loss.
    """
    net.train()
    running_loss, total = 0.0, 0
    for _ in tqdm(range(epochs)):
        running_loss = 0.0
        total, batch_cnt = 0, 0
        for i, (data, labels) in enumerate(train_loader):
            data, labels = data.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = net(data)
            loss = criterion(net(data), labels)
            running_loss += loss.item()
            total += labels.size(0)
            loss.backward()
            optimizer.step()

            # Break if we have exceeded the upper limit
            # On training batches for a given round
            # Simulate enumerate counting for train/test parity
            if batch_cnt > max_train_batches_per_epoch:
                break
            batch_cnt += 1
    return running_loss / total

In [ ]:
def test_FEMNIST(
    net: Module,
    test_loader: DataLoader,
    device: str,
    criterion: Module,
) -> Tuple[float, float]:
    """Validate the network on a test set.

    Args:
        net (Module): generic module object describing the network to test.
        test_loader (DataLoader): dataloader to iterate during the testing.
        device (str):  device name onto which perform the computation.
        criterion (Module): generic module describing the loss function.

    Returns:
        Tuple[float, float]: couple of average test loss and average accuracy on the test set.
    """
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for i, (data, labels) in enumerate(
            tqdm(test_loader, total=min(max_test_batches_per_epoch, len(test_loader)))
        ):
            data, labels = data.to(device), labels.to(device)
            outputs = net(data)

            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Break if we have exceed the upper limit
            # On testing batches
            if i > max_test_batches_per_epoch:
                break

    accuracy = correct / total
    return loss, accuracy

In [ ]:
class CompleteSimpleClient(SimpleClient):
    def _load_dataset(self, name, config: Dict[str, Any]) -> Dataset:
        full_file: Path = self.partition_dir / self.cid
        return load_FEMNIST_dataset(mapping=full_file, name=name)

    def _train(
        self, net: Module, train_loader: DataLoader, config=Dict[str, Any]
    ) -> float:
        # Notice the usage of the config dict to obtain training
        # parameters for a given client
        return train_FEMNIST(
            net=net,
            train_loader=train_loader,
            epochs=config["epochs"],
            device=self.device,
            optimizer=torch.optim.AdamW(
                net.parameters(),
                lr=config["client_learning_rate"],
                weight_decay=config["weight_decay"],
            ),
            criterion=torch.nn.CrossEntropyLoss(),
        )

    def _test(
        self, net, test_loader: DataLoader, config: Dict[str, Any]
    ) -> Tuple[float, float]:
        return test_FEMNIST(
            net=net,
            test_loader=test_loader,
            device=self.device,
            criterion=torch.nn.CrossEntropyLoss(),
        )

This client can now encapsulate any partition of the FEMNIST dataset and train and test on it. To put it into action, we shall define a small CNN taken from the [60 minute PyTorch tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#define-a-convolutional-neural-network) and the training/testing configurations used by the fit/evaluate functions of the client.

In [ ]:
##  Define a simple CNN


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(256, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 62)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

The following global variables will be set to standardise training and testing across experiments. This means fixing:
- The starting initialised model.
- Local train/test parameters.

In [ ]:
# All experiments will have the same initialisation.
# All differences in performance will come from training
def get_network_generator():
    untrained_net: Net = Net()

    def generated_net():
        return deepcopy(untrained_net)

    return generated_net


network_generator = get_network_generator()

centralized_train_config: Dict[str, Any] = {
    "epochs": 5,
    "batch_size": 32,
    "client_learning_rate": 0.01,
    "weight_decay": 0.001,
    "num_workers": 0,
}

test_config: Dict[str, Any] = {
    "batch_size": 32,
    "num_workers": 0,
}

In [ ]:
# Instantiate the centralised client and model
centralized_client = CompleteSimpleClient(cid=0, partition_dir=centralized_partition)
centralized_net = network_generator()

In [ ]:
# Fit the centralised model
centralized_net, *rest = fit_client_seeded(
    centralized_client, params=centralized_net, conf=centralized_train_config
)
print(rest)

In [ ]:
# Test the trained centralised model
result = centralized_client.evaluate(net=centralized_net, config=test_config)
print(result)

**Question 4 (Part II ✅ | Part III/MPhil ✅):**

(This is meant to be a conceptual question. You should provide written answers for this. **No more than 3 sentences**. **No code** is needed)

1. Read about [data-parallelism](https://d2l.ai/chapter_computational-performance/multiple-gpus.html), if we were to train the centralized model in a data-parallel fashion how often would we need to communicate between the model replicas?

# END OF PART I:

Continue to the part 2 notebook.

(c) 2024 Alexandru-Andrei Iacob, Lorenzo Sani